In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


First, let's see the train data and the test data.

In [5]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


We are going to use a random forest regressor. We will train the model with train_data, using the columns concerning ticket class ('pclass'), sex ('sex'), number of sister or spouses aboard the Titanic ('SibSp') and number of parents or children aboard the Titanic ('Parch'). Then, we will predict whether the passengers from test_data survived (indicated by 1) or not (indicated by 0). As the predictions will be a pandas series formed by values between 0 and 1, we will round each value to 0 or 1.

In [7]:
def sex_01(x):
    if x == 'male':
        return 0
    if x == 'female':
        return 1

test_data.Sex = test_data.Sex.apply(lambda x: sex_01(x))

y = train_data.Survived
features = ["Pclass", "Sex", "SibSp", "Parch"]

X = train_data[features]
X.Sex = X.Sex.apply(lambda x: sex_01(x))

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


It would be interesting if we validated the model first by calculating the mean absolute error. We need to split the data and fit the model with only a part to predict the rest. 

In [9]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0)

from sklearn.ensemble import RandomForestRegressor

# Define the model. Set random_state to 1
forest = RandomForestRegressor(random_state=1)

# fit your model
forest.fit(train_X, train_y)

prediction_forest = forest.predict(val_X)
prediction_forest = pd.Series(prediction_forest)

def round(x):
    if x < 0.8:
        return 0
    if x >= 0.2:
        return 1
    
prediction_forest = prediction_forest.apply(lambda x: round(x))

from sklearn.metrics import mean_absolute_error
# Calculate the mean absolute error of your Random Forest model on the validation data
forest_mae = mean_absolute_error(val_y, prediction_forest)

print("Validation MAE for Random Forest Model: {}".format(forest_mae))

Validation MAE for Random Forest Model: 0.2062780269058296


In [92]:
forest.fit(X,y)
X_test = pd.get_dummies(test_data[features])
prediction_forest_2 = forest.predict(X_test)
prediction_forest_2 = pd.Series(prediction_forest_2)
prediction_forest_2 = prediction_forest_2.apply(lambda x: round(x))

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': prediction_forest_2})
output.to_csv('submission.csv', index=False)
output

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
